In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_MA_journal
@pyimport GLS_Apr_weekday_PM_journal

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in transpose(::Ptr{PyCall.PyObject_struct}) at ./deprecated.jl:770
 in transpose_f!(::Base.#transpose, ::Array{Ptr{PyCall.PyObject_struct},2}, ::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:369
 in transpose(::Array{Ptr{PyCall.PyObject_struct},2}) at ./arraymath.jl:407
 in copy(::PyCall.PyArray{Ptr{PyCall.PyObject_struct},2}) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:337
 in convert(::Type{Array{Ptr{PyCall.PyObject_struct},N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:453
 in convert(::Type{Array{PyCall.PyObject,N}}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/numpy.jl:484
 in convert(::Type{PyCall.PyAny}, ::PyCall.PyObject) at /home/jzh/.julia/v0.5/PyCall/src/conversions.jl:806
 in #55 at ./<missing>:0 [inlined]
 in next at ./generator.jl:26 [inlined]
 in collect_to!(::Array{Expr,1}, ::Base.Generator{Array{Tuple{AbstractString,PyCall.PyObject},1},PyCall.##55#59}, ::Int64, ::Int64) at

In [2]:
numNodes = Compute_Jacobian_MA_journal.numNodes;
numLinks = Compute_Jacobian_MA_journal.numLinks;
numODpairs = Compute_Jacobian_MA_journal.numODpairs;
numZones = Compute_Jacobian_MA_journal.numZones
od_pairs = Compute_Jacobian_MA_journal.od_pairs;
link_list_js = Compute_Jacobian_MA_journal.link_list_js;
link_length_list = Compute_Jacobian_MA_journal.link_length_list;

flow_observ = GLS_Apr_weekday_PM_journal.x_

258×30 Array{Float64,2}:
 1393.18     806.378    762.959        …  1122.05    1513.32     356.411 
 1245.77    1047.31    1023.88            1058.72    1286.13     737.719 
 2175.98    1580.96    1289.54            1310.68    1660.56    1011.89  
 2206.51    1358.68    1100.85            1375.95    2022.21     856.967 
   27.0447   279.236    279.358            114.702    182.177    319.842 
  143.919    260.583    207.122        …   112.767     47.7074    93.4537
  128.953     91.0833    77.1364           105.58     127.956     72.177 
  128.953     91.0833    77.1364           105.58     127.956     72.177 
 1381.89    1661.69    1435.11            1024.77    1282.9     1395.51  
 1061.96    1922.71    1849.59             893.572    782.416   2032.03  
   85.5516   284.675    357.062        …    50.566     29.8585   369.656 
  258.077    264.587    203.507            118.433    303.162    114.442 
  322.493    399.126    250.17             333.773    473.913    174.446 
    ⋮        

In [3]:
include("../Julia_files/initia_data.jl");
include("prepare_data_MA_journal.jl");
include("extract_data_MA_journal.jl");
include("../Julia_files/tap_MSA.jl");
include("../Julia_files/inverseVI.jl");
include("../Julia_files/demands_adjustment_gradi.jl");

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/prepare_data_MA_journal.jl:183
 in include_from_node1(::String) at ./loading.jl:488
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading /home/jzh/Research/Traffic/InverseVIsTraffic/08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/extract_data_MA_journal.jl, in expression starting on line 10
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in furInfo() at /home/jzh/Research/Traffic/InverseVIsTraffic/08_de

In [4]:
# demandsDict_ = readall("../08_develop_new_OD_demand_estimator_MA_Dijkstra_uni_class_Apr_PM_journal/results/demandsDict.json");
# demandsDict_ = JSON.parse(demandsDict_);

In [5]:
# demandsDict_

In [6]:
# demandsDict__ = demandsDict_["24"]

# _demandsDict__ = Dict()

# for key_ in keys(demandsDict__)
#     key = (int(split(split(key_, ",")[1], "(")[2]),int(split(split(key_, ",")[2], ")")[1]))
#     _demandsDict__[key] = demandsDict__[key_]
# end

In [7]:
# _demandsDict__

In [8]:
# demandsDict[0] = _demandsDict__;
# demandsDict[1] = _demandsDict__;

In [9]:
demandsDict[0] == demandsDict[1]

true

In [10]:
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]));

In [11]:
demandsDiffDict[1]

0.0

In [12]:
function demandsDictFixed(day)
#     day = 4  # day of April

    # observed flow vector
    xl = flow_observ[:, day]

    tapFlows = Dict{}()

    for i = 1:length(ta_data.start_node)
        key = (ta_data.start_node[i], ta_data.end_node[i])
        tapFlows[key] = xl[i]
    end

    tapFlowVect = xl;

    # get observed flow vector (corresponding to ground truth demands and ground truth costs)
    tapFlowDicDict[0], tapFlowVecDict[0] = tapFlows, tapFlowVect;

    # get arcs data corresponding to ground truth demands and flows
    arcsDict[0] = observFlow("./benchmark_data/journal_net_Apr_PM.txt", tapFlowDicDict[0]);

    arcsDict[0]

    deg = 8
    c = 0.5
    lam = 10000.0

    coeffs_dict_Apr_PM_ = readall("../temp_files/coeffs_dict_Apr_PM.json")
    coeffs_dict_Apr_PM_ = JSON.parse(coeffs_dict_Apr_PM_)
    fcoeffs = coeffs_dict_Apr_PM_["($(deg),$(c),$(lam),1)"]

    # fcoeffs = [1, 0, 0, .15]

    demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
    objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[1], fcoeffs);

    # get initial flow vector (corresponding to initial demands)
    tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], fcoeffs);

    demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

    # get arcs data corresponding to initial demands and flows
    arcsDict[1] = observFlow("./benchmark_data/journal_net_Apr_PM.txt", tapFlowDicDict[1]);

    linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], fcoeffs);

    linkCostDicDict[1]["0"], link_length_list[1]

    jacobiSpiessDict[1] = Compute_Jacobian_MA_journal.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                                  link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

    # maximum number of iterations
    N = 100;  

    # Armijo rule parameters
    rho = 2;
    M = 10;

    # search direction parameter
    epsilon_1 = 0;

    # stop criterion parameter
    epsilon_2 = 1e-20;

    for l = 1:N

        jacobDict[l] = jacobiSpiessDict[l]

        descDirecDict[l] = descDirec(tapFlowVecDict[l], tapFlowVecDict[0], jacobDict[l], numODpairs, numLinks);

        demandsVecDict[l] = demandsDicToVec(demandsDict[l]);

        searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

        thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

        demandsVecDict[l+1] = similar(demandsVecDict[0]);

        demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
            demandsVecDict[0], fcoeffs, searchDirecDict[l], thetaMaxDict[l], rho, M);
        
        demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);

        tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], fcoeffs);

        arcsDict[l+1] = observFlow("./benchmark_data/journal_net_Apr_PM.txt", tapFlowDicDict[l+1]);

        linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], fcoeffs);

        jacobiSpiessDict[l+1] = Compute_Jacobian_MA_journal.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                                  link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);

        demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);

        # stopping criterion
        if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
            break
        end
        
        println("iteration $(l) finished...")

    end

    # normalize objective function value
    for l = 1:(length(objFunDict))
        norObjFunDict[l] = objFunDict[l] / objFunDict[1];
    end


    outfile = open("./results/demandsVecDict$(day)_journal.json", "w")

    JSON.print(outfile, demandsVecDict)

    close(outfile)

    outfile = open("./results/demandsDict$(day)_journal.json", "w")

    JSON.print(outfile, demandsDict)

    close(outfile)

    outfile = open("./results/tapFlowDicDict$(day)_journal.json", "w")

    JSON.print(outfile, tapFlowDicDict)

    close(outfile)

    outfile = open("./results/tapFlowVecDict$(day)_journal.json", "w")

    JSON.print(outfile, tapFlowVecDict)

    close(outfile)

    demandsDict[length(demandsDict)-1]

    demandsDict_ = Dict{}()

    for key in keys(demandsDict[length(demandsDict)-1])
        demandsDict_[key] = demandsDict[length(demandsDict)-1][key] 
    end

    outfile = open("./results/demandsDictFixed$(day)_journal.json", "w")

    JSON.print(outfile, demandsDict_)

    close(outfile)
end

demandsDictFixed (generic function with 1 method)

In [13]:
# week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
week_day_Apr_list = [2]

for day in week_day_Apr_list
    demandsDictFixed(day)
end

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in demandsDictFixed(::Int64) at ./In[12]:28
 in macro expansion; at ./In[13]:5 [inlined]
 in anonymous at ./<missing>:?
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading In[13], in expression starting on line 4


LoadError: LoadError: MethodError: no method matching objF(::Array{Float64,1}, ::Array{Any,1})
Closest candidates are:
  objF(::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any) at /home/jzh/Research/Traffic/InverseVIsTraffic/Julia_files/demands_adjustment_gradi.jl:135
while loading In[13], in expression starting on line 4